# Model Iteration 1 
To start, we are going to implement a new kind of model for the data we found when exploring through the kaggle scripts: the Bernoulli Distribution. Then we will iterate, and improve, the model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv("train.csv")

/home/mackenzie/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


First things first, we are going to clean days of week to be values ranging from 0 - 6 (Mon - Sun) using a dictionary and mapping. This is another new method of replacement that so far has been pretty advantageous for changing these strings to integers/floats that can be analyzed

In [2]:
dow = {
    "Monday" : 0,
    "Tuesday" : 1,
    "Wednesday" : 2,
    "Thursday" : 3,
    "Friday" : 4,
    "Saturday" : 5,
    "Sunday" : 6
}
data["DOW"] = data.DayOfWeek.map(dow)

Next step is similar, but now with Police Districts so we can just deal with ints instead of strings. Data exploration showed these districts are good measures of district-based crime.

In [3]:
pds = {
    "SOUTHERN" : 0,
    "MISSION" : 1,
    "NORTHERN" : 2,
    "BAYVIEW" : 3,
    "CENTRAL" : 4,
    "TERNDERLOIN" : 5,
    "INGLESIDE" : 6,
    "TARAVAL" : 7,
    "PARK" : 8,
    "RICHMOND" : 9
}
data["pds"] = data.PdDistrict.map(pds)
# for crimes without PD, use "Other" : 10
data["pds"] = data["pds"].fillna(10)

Next, we are going to use X and Y coordinates as they are pretty continuous indicators of location (which is helpful in predicting the incident's category). There are some outlying data points, so let's clean that up. First, we check for those outlying data points (which we happen to know are at the north pole).

In [4]:
for val in data.Y:
    if val == 90.0:
        print val
        
for val in data.X:
    if val == -120.5:
        print val

90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
90.0
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5
-120.5


Then we replace those points for more reasonable numbers. Running the code below, then the code above will produce 0 outlying values (because we got rid of them).

In [5]:
data.X.replace(-120.5, data["X"].median(), inplace = True)
data.Y.replace(90, data["Y"].median(), inplace = True)

The data cleaning train has no brakes. Now we are going to clean up date/time. We're using year, month, and hour because our data explorations revealed there's a good chunk of dependance on crime rates throughout the years, seasonally, and at different hours of the day.

In [6]:
def parse_date(Dates):
    """ Convert a date in YYYY-MM-DD HH:MM:SS to a tuple
        containing year, month, day, and hours each expressed
        as an integer. Used from Paul Ruvolo's example in bikeshare kaggle dataset

        >>> parse_date("2014-04-05 14:00:00")
        (2014, 4, 5, 14)
    """
    return int(Dates[0:4]), int(Dates[5:7]), int(Dates[8:10]), int(Dates[11:13])

# "now we use the lambda functions" - Mack
data["Year"] = data.Dates.apply(lambda x: parse_date(x)[0])
data["Month"] = data.Dates.apply(lambda x: parse_date(x)[1])
data["Hour"] = data.Dates.apply(lambda x: parse_date(x)[3])

And so now we are going to do the predictive thingamajig. We drew inspiration from this script (https://www.kaggle.com/sonuk7/sf-crime/prediction-with-bernoulinb/code ).Bernoulli distributions deal with discrete variables where we have a two-point distribution. the BernoulliNB method gives us the probability of an incident fitting a Category when the probability is trained with our input columns to be better than 50-50. We're using this because this data set deals a lot with categorical variables instead of a bunch of integers that could be continous, like in the warm-up project. 

In [7]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import cross_validation

cats = data.Category.values
cleanData = data.drop(["Address","Category","Dates","Descript","X", "Y","Resolution", "DayOfWeek", "PdDistrict"], axis=1)

model = BernoulliNB()
model.fit(cleanData.dropna(), cats)

scores = cross_validation.cross_val_score(model, cleanData, data["Category"], cv = 3)
print scores

[ 0.19976153  0.19864289  0.19928998]


Wow that's a bad score :\ I think? Not sure really. Let's generate a test file starting with clean test data to mimic our cleaned training data



In [8]:
import gzip, csv

test = pd.read_csv('test.csv')
test["DOW"] = test.DayOfWeek.map(dow)

test["pds"] = test.PdDistrict.map(pds)
# for crimes without PD, use "Other" : 10
test["pds"] = test["pds"].fillna(10)

test.X.replace(-120.5, test["X"].median(), inplace = True)
test.Y.replace(90, test["Y"].median(), inplace = True)

test["Year"] = test.Dates.apply(lambda x: parse_date(x)[0])
test["Month"] = test.Dates.apply(lambda x: parse_date(x)[1])
test["Hour"] = test.Dates.apply(lambda x: parse_date(x)[3])
                                
idx = test.Id.values

cleanTest = test.drop(["Id","Address","Dates","X", "Y", "DayOfWeek", "PdDistrict"], axis=1)

model = BernoulliNB()
model.fit(cleanData.dropna(), cats)

predicted = model.predict_proba(cleanTest)
#labels =  "ARSON,ASSAULT,BAD CHECKS,BRIBERY,BURGLARY,DISORDERLY CONDUCT,DRIVING UNDER THE INFLUENCE,DRUG/NARCOTIC,DRUNKENNESS,EMBEZZLEMENT,EXTORTION,FAMILY OFFENSES,FORGERY/COUNTERFEITING,FRAUD,GAMBLING,KIDNAPPING,LARCENY/THEFT,LIQUOR LAWS,LOITERING,MISSING PERSON,NON-CRIMINAL,OTHER OFFENSES,PORNOGRAPHY/OBSCENE MAT,PROSTITUTION,RECOVERED VEHICLE,ROBBERY,RUNAWAY,SECONDARY CODES,SEX OFFENSES FORCIBLE,SEX OFFENSES NON FORCIBLE,STOLEN PROPERTY,SUICIDE,SUSPICIOUS OCC,TREA,TRESPASS,VANDALISM,VEHICLE THEFT,WARRANTS,WEAPON LAWS".split(',')
labels =['Id']
for i in model.classes_:
    labels.append(i)
with gzip.open('bernoulinb.csv.gz', 'wb') as outf:
    fo =csv.writer(outf, lineterminator = '\n' )
    fo.writerow(labels)
    
    for i, pred in enumerate(predicted):
        fo.writerow([i] + list(pred))
#predictions = model.predict(cleanTest)

#submission = pd.DataFrame({
#        "Id" : cleanTest["Id"],
#        "Category" : predictions
#    })

#submission.to_csv("CYOA1.csv", index=False)

This submission got us a 2.66746 on our first try, yay! That means our abouve test scores were actually really good. We're bringing this to class to discuss on Tuesday. The sample submission got a 32 and is ranked at 1203 while we ranked at 818. 